In [28]:
# prepare library

import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [29]:
# uploading first data set (training)

train = pd.read_csv('./train.csv')
train.head()

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,C6,C7,C8,N1,C9,N2,N3,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
0,-1,37,19274330,-1,0,2569487005,1830830742,4228834423,652481754,2967512603,3624739744,2303407174,2206061889,0,4287642944,0,0,0,0,0,0,0,0,0,419316267,3617851689,4220791646
1,-1,55,19274334,-1,0,3191875643,403381353,1238330876,652481754,2967512603,179712055,2303407174,157533148,1,3272771221,0,1,0,0,0,0,1,0,0,1968545380,3831440054,1213938795
2,-1,114,19274353,-1,0,2634604934,403381353,1928325921,652481754,2967512603,1607619193,2303407174,3588031319,0,3329757940,0,0,0,0,0,0,0,0,0,1405802835,2379633508,1284343215
3,-1,128,19274358,-1,0,2569487005,403381353,296915027,652481754,2967512603,10212495,2303407174,3588031319,0,3427046189,0,0,0,0,0,0,0,0,0,2299319102,3617851689,4220791646
4,-1,164,19274367,-1,0,2569487005,403381353,681762206,652481754,2967512603,580883881,2303407174,157533148,0,2088887026,0,0,0,0,0,0,0,0,0,1306075469,3617851689,4220791646


In [30]:
train.columns

Index(['label', 'id', 'visitTime', 'purchaseTime', 'hour', 'C1', 'C2', 'C3',
       'C4', 'C5', 'C6', 'C7', 'C8', 'N1', 'C9', 'N2', 'N3', 'N4', 'N5', 'N6',
       'N7', 'N8', 'N9', 'N10', 'C10', 'C11', 'C12'],
      dtype='object')

In [37]:
# handle categorical data
col = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','id']
for name in col:
    train[name] = train[name].astype('object')

In [38]:
# strasify predicted data ('label')

train_clean = train.drop(columns=['label','id','visitTime','purchaseTime','C1','C3','C10']).values

X = train_clean.copy()
y = np.array(label.copy())

In [42]:
df = train.copy()
train_data = train[['N1','N2','N3','N4','N5','N6','N7','N8','N9','N10','C2','C4','C7']] # 'C4','C7'
train_data = pd.get_dummies(train,drop_first=True)
train_data.shape

(31428, 85768)

In [ ]:
# start to re-sampling
id_toTrain = np.array([np.where(Y_train==i)[0] for i in range(nb_class)])

size_max = [len(id_toTrain[i]) for i in range(nb_class)]
print(f"Before resampling we the distribution is as follows [non-sale, sale]: {size_max}")

blc = 150
for i in range(len(size_max)):
    if size_max[i] > blc:
        size_max[i] = int(blc*(np.log10(size_max[i]/blc)+1))
    else:
        size_max[i] = int(blc/(np.log10(blc/size_max[i])+1))

print(f"After resampling we the distribution is as follows [non-sale, sale]: {size_max}")

for i in range(nb_class):
    if len(id_toTrain[i]) > size_max[i]:
        id_toTrain[i], tmp = train_test_split(id_toTrain[i], test_size=1-size_max[i]/len(id_toTrain[i]))
    else:
        id_toTrain[i] = np.concatenate((id_toTrain[i], id_toTrain[i][np.random.randint(len(id_toTrain[i]), size=int(size_max[i]-len(id_toTrain[i])))]))
id_toTrain = np.concatenate(id_toTrain)
X_toTrain = X_train[id_toTrain]
Y_toTrain = Y_train[id_toTrain]


In [44]:
len(train['label'])

31428

In [45]:
X = train.loc[:, train.columns != 'label']
y = train.loc[:, train.columns == 'label']
y = y.astype('int')
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['label'])

In [46]:
print("Length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['label']==-1]))
print("Number of subscription",len(os_data_y[os_data_y['label']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['label']==-1])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['label']==1])/len(os_data_X))

Length of oversampled data is  43914
Number of no subscription in oversampled data 21957
Number of subscription 21957
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


In [47]:
train = pd.concat([os_data_y, os_data_X], axis=1)
train.head()

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,C6,C7,C8,N1,C9,N2,N3,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
0,-1,103743,19301822,-1,7,2.56949e+09,4.03381e+08,3.47535e+09,6.52482e+08,2.96751e+09,2.08799e+09,2.30341e+09,3.58803e+09,0,1.26614e+09,0,0,0,0,0,0,0,0,0,3.54393e+09,3.61785e+09,4.22079e+09
1,-1,560491,19339276,-1,18,2.05547e+09,1.23598e+09,4.26136e+09,6.52482e+08,2.11677e+09,5.71766e+08,2.30341e+09,2.37495e+09,0,2.22668e+09,0,0,0,0,0,0,0,0,0,3.94503e+09,3.83144e+09,1.21394e+09
2,-1,367412,19321168,-1,13,2.56949e+09,2.01871e+09,4.35761e+08,6.52482e+08,2.96751e+09,3.01712e+09,2.30341e+09,3.58803e+09,0,2.71109e+09,0,0,0,0,0,0,0,0,0,2.33689e+09,3.61785e+09,4.22079e+09
3,-1,155153,19305580,-1,8,2.35941e+09,4.03381e+08,2.00007e+08,6.52482e+08,2.96751e+09,2.7125e+09,2.30341e+09,2.20606e+09,0,3.8598e+08,0,0,0,0,0,0,0,0,0,3.72524e+09,3.83144e+09,2.54898e+09
4,-1,61015,19297622,-1,6,2.56949e+09,2.5412e+09,2.59372e+09,2.30817e+09,2.96751e+09,3.17864e+09,2.30341e+09,3.58803e+09,0,3.87475e+08,0,0,0,0,0,0,0,0,0,2.68126e+09,3.61785e+09,4.22079e+09


In [48]:
train_final_vars = train.columns.values.tolist()
y = ['label']
X = [i for i in train_final_vars if i not in y]

logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)
print(columns)

[ True  True  True False  True  True  True  True  True  True  True  True
 False  True False  True  True False False False  True  True  True  True
  True  True]
[1 1 1 3 1 1 1 1 1 1 1 1 5 1 6 1 1 4 2 7 1 1 1 1 1 1]
Index(['id', 'visitTime', 'purchaseTime', 'hour', 'C1', 'C2', 'C3', 'C4', 'C5',
       'C6', 'C7', 'C8', 'N1', 'C9', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8',
       'N9', 'N10', 'C10', 'C11', 'C12'],
      dtype='object')


In [49]:
# select only significant varialbes to the targeted value
cols = ['N1', 'N3', 'N8','N9', 'N10']

X = os_data_X[cols]
y = os_data_y['label']
os_data_y['label'].replace({-1: 0}, inplace=True)

logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.359678
         Iterations 12
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.481     
Dependent Variable: label            AIC:              31599.7746
Date:               2020-03-22 21:54 BIC:              31643.2245
No. Observations:   43914            Log-Likelihood:   -15795.   
Df Model:           4                LL-Null:          -30439.   
Df Residuals:       43909            LLR p-value:      0.0000    
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     12.0000                                      
-------------------------------------------------------------------
          Coef.    Std.Err.      z       P>|z|     [0.025    0.975]
-------------------------------------------------------------------
N1       -1.2048     0.0286   -42.0642   0.0000   -1.2609   -1.1487
N3        0.3731     0.0225    16.5872   0.0000    0.3290

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
# start predicting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression: 0.94


In [61]:
# prepare to aggregate data
pred_list = y_pred.tolist()

X_with_prediction = X
X_with_prediction ["Predicted score"] = pred_list

train['id']

0        103743
1        560491
2        367412
3        155153
4         61015
          ...  
43909    386517
43910    458182
43911    328534
43912    115831
43913    398585
Name: id, Length: 43914, dtype: object

In [62]:
Export = pd.concat([train['id'], X_with_prediction['Predicted score']], axis=1)

In [65]:
Export.to_csv('/Users/macbook/Desktop/ML with Python/Assignment 2-3/Final_Assignment-Tanasit.csv', index = False)